<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-19 09:34:44
-------------------
qualified stocks: 95
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.11 L
Current:  1.29 C
-------------------
Today PnL: 71.12 K (0.56%)
Current PnL: -15.08 L (-10.7%)
CY Booked + Current PnL: -11.95 L (-8.48%)
-------------------
Total profit:  4.49 L
Total loss:  -19.57 L
-------------------
Total Booked + Current PnL: 15.03 L (13.18%)
Total Booked PnL: 30.11 L (26.4%)
Curr Year Booked PnL: 3.13 L (2.43%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 77.79 L (60.4%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.39 C (108.13%)
Deployed:  1.14 C
Current:  1.29 C
CAGR/XIRR %: 8.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELIANCE,1288.53,1526.00,6.12,X-LC,82.11,228847.0,25259.0,12266.0,-0.55,12.41,5.36,18.43,157.0,2.06,1.81,24.66,XY25,NTT,REFINERIES
69,SBILIFE,1496.49,1924.99,0.20,H-LC,73.68,148915.0,23210.0,12777.0,-0.03,18.46,8.58,28.63,147.0,1.82,1.18,32.66,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-16.01,X-LC,42.11,93030.0,17908.0,14252.0,0.32,23.84,15.32,42.81,120.0,1.26,0.73,16.67,X40,BTT,CHEMICALS
6,ANGELONE,2328.67,3033.00,11.80,H-SC,88.42,182162.0,30798.0,14974.0,0.07,20.35,8.22,30.25,111.0,2.06,1.44,44.18,X40N,NTT,FINANCE
70,SBIN,760.30,863.00,-13.45,M-LC,24.21,210874.0,9394.0,17819.0,0.46,4.66,8.45,13.51,60.0,0.53,1.66,17.85,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TITAGARH,1097.23,1548.0,-3.25,H-SC,98.95,170074.0,-21941.0,100820.0,6.46,-11.43,59.28,41.08,95.0,-0.22,1.34,43.95,XY24,NTT,ENGINEERING
89,VALIANTORG,512.64,838.0,-337.06,H-SC,55.79,100893.0,-32393.0,116985.0,3.56,-24.30,115.95,63.47,161.0,-0.28,0.80,69.12,XR,NTT,CHEMICALS
51,LUXIND,1448.19,2442.0,47.89,M-SC,78.95,153340.0,15762.0,78648.0,3.18,11.46,51.29,68.62,79.0,0.20,1.21,32.94,X40N,NTT,TEXTILES
1,AARTIIND,487.04,919.0,41.53,M-SC,91.58,81312.0,-24.0,72164.0,2.58,-0.03,88.75,88.69,159.0,-0.00,0.64,37.12,XR,NTT,CHEMICALS
0,5PAISA,564.75,810.0,148.80,H-SC,44.21,134320.0,-52612.0,133796.0,2.57,-28.15,99.61,43.43,183.0,-0.39,1.06,28.50,X40N,BTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HONAUT,42646.15,58518.52,-21.77,X-MC,77.89,114525.0,-13413.0,61030.0,-1.66,-10.48,53.29,37.22,73.0,-0.22,0.90,16.92,X40N,ATH,ELECTRICAL
41,INFY,1432.81,2275.00,-12.42,X-LC,71.58,218425.0,19264.0,97811.0,-1.16,9.67,44.78,58.78,10.0,0.20,1.72,14.04,X40,BTT,IT
59,RAJESHEXPO,517.65,583.00,1849.49,M-SC,32.63,53201.0,-83976.0,101295.0,-1.14,-61.22,190.40,12.62,195.0,-0.83,0.42,32.31,X40N,NTT,JEWELLERY
53,MEDANTA,1022.60,1486.00,2.53,X-MC,5.26,180728.0,25293.0,45146.0,-1.01,16.27,24.98,45.32,201.0,0.56,1.43,22.97,XY24,NTT,HEALTHCARE
46,KANSAINER,299.63,340.00,-66.21,H-SC,18.95,235350.0,-34317.0,70652.0,-0.80,-12.73,30.02,13.47,162.0,-0.49,1.86,17.80,XY24,NTT,PAINTS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,776.02,1065.00,-51.41,M-LC,93.68,195186.0,-10459.0,87033.0,0.80,-5.09,44.59,37.24,1.0,-0.12,1.54,27.05,XY24,NTT,AUTO
38,INDIAMART,2311.97,4911.36,-52.40,H-SC,47.37,103017.0,1290.0,113082.0,-0.38,1.27,109.77,112.43,2.0,0.01,0.81,21.58,AR,ATH,MISC
2,ABB,5319.37,8953.12,-35.79,H-LC,53.68,146975.0,13991.0,76853.0,1.01,10.52,52.29,68.31,4.0,0.18,1.16,19.85,AR,ATH,ELECTRICAL
82,TCS,3928.29,4998.00,-15.52,X-LC,62.11,190847.0,-21281.0,79049.0,-0.76,-10.03,41.42,27.23,5.0,-0.27,1.51,9.37,X40,BTT,IT
49,LICI,777.74,983.00,-13.28,H-LC,54.74,192506.0,18292.0,27682.0,0.28,10.50,14.38,26.39,6.0,0.66,1.52,17.86,XY25,NTT,INSURANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DMART,3484.00,5112.00,-17.59,X-LC,15.79,101419.0,17803.0,21268.0,0.86,21.29,20.97,46.73,146.0,0.84,0.80,24.14,AR,NTT,FMCG
56,PIDILITIND,2504.06,3576.00,-16.01,X-LC,42.11,93030.0,17908.0,14252.0,0.32,23.84,15.32,42.81,120.0,1.26,0.73,16.67,X40,BTT,CHEMICALS
6,ANGELONE,2328.67,3033.00,11.80,H-SC,88.42,182162.0,30798.0,14974.0,0.07,20.35,8.22,30.25,111.0,2.06,1.44,44.18,X40N,NTT,FINANCE
57,POLYCAB,5005.92,7541.35,9.37,H-MC,85.26,171080.0,30914.0,40084.0,0.25,22.06,23.43,50.65,55.0,0.77,1.35,30.57,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-0.40,M-LC,86.32,192526.0,41320.0,24335.0,-0.15,27.33,12.64,43.43,37.0,1.70,1.52,33.45,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,47.89,M-SC,78.95,153340.0,15762.0,78648.0,3.18,11.46,51.29,68.62,79.0,0.20,1.21,32.94,X40N,NTT,TEXTILES
6,ANGELONE,2328.67,3033.00,11.80,H-SC,88.42,182162.0,30798.0,14974.0,0.07,20.35,8.22,30.25,111.0,2.06,1.44,44.18,X40N,NTT,FINANCE
57,POLYCAB,5005.92,7541.35,9.37,H-MC,85.26,171080.0,30914.0,40084.0,0.25,22.06,23.43,50.65,55.0,0.77,1.35,30.57,X40N,ATH,CABLES
33,HEROMOTOCO,4311.81,6039.03,1.43,H-MC,84.21,154679.0,3766.0,56690.0,1.71,2.50,36.65,40.06,14.0,0.07,1.22,26.52,AR,ATH,AUTO
28,GILLETTE,8109.44,10330.69,0.27,H-SC,76.84,147815.0,9955.0,27804.0,0.99,7.22,18.81,27.39,68.0,0.36,1.17,29.39,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-2.22,H-SC,17.89,109936.0,7932.0,37092.0,0.91,7.78,33.74,44.14,22.0,0.21,0.87,25.28,XR,ATH,MISC
33,HEROMOTOCO,4311.81,6039.03,1.43,H-MC,84.21,154679.0,3766.0,56690.0,1.71,2.50,36.65,40.06,14.0,0.07,1.22,26.52,AR,ATH,AUTO
94,WIPRO,243.46,420.00,-8.35,M-LC,61.05,157288.0,6343.0,103118.0,-0.24,4.20,65.56,72.51,52.0,0.06,1.24,18.42,XR,NTT,IT
12,BANKINDIA,113.49,190.00,-33.09,M-MC,13.68,132213.0,3061.0,84008.0,1.62,2.37,63.54,67.42,41.0,0.04,1.04,28.59,XR,NTT,BANKS
38,INDIAMART,2311.97,4911.36,-52.40,H-SC,47.37,103017.0,1290.0,113082.0,-0.38,1.27,109.77,112.43,2.0,0.01,0.81,21.58,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VALIANTORG,512.64,838.00,-337.06,H-SC,55.79,100893.0,-32393.0,116985.0,3.56,-24.30,115.95,63.47,161.0,-0.28,0.80,69.12,XR,NTT,CHEMICALS
86,UJJIVANSFB,52.77,60.00,40.02,M-SC,31.58,121068.0,-21411.0,40933.0,0.56,-15.03,33.81,13.70,214.0,-0.52,0.96,42.89,X40N,NTT,BANKS
12,BANKINDIA,113.49,190.00,-33.09,M-MC,13.68,132213.0,3061.0,84008.0,1.62,2.37,63.54,67.42,41.0,0.04,1.04,28.59,XR,NTT,BANKS
69,SBILIFE,1496.49,1924.99,0.20,H-LC,73.68,148915.0,23210.0,12777.0,-0.03,18.46,8.58,28.63,147.0,1.82,1.18,32.66,AR,ATH,INSURANCE
87,UNIONBANK,123.87,163.00,-12.58,M-LC,48.42,151960.0,11120.0,33370.0,0.47,7.90,21.96,31.59,48.0,0.33,1.20,32.00,XY24,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VIPIND,488.80,718.00,-887.19,H-SC,100.00,75026.0,-20290.0,64980.0,0.92,-21.29,86.61,46.89,82.0,-0.31,0.59,47.98,X40N,NTT,MISC
83,TITAGARH,1097.23,1548.00,-3.25,H-SC,98.95,170074.0,-21941.0,100820.0,6.46,-11.43,59.28,41.08,95.0,-0.22,1.34,43.95,XY24,NTT,ENGINEERING
79,TANLA,917.30,1963.11,-38.20,H-SC,97.89,151902.0,-87513.0,360463.0,1.83,-36.55,237.30,114.01,20.0,-0.24,1.20,39.32,AR,ATH,IT
93,WHIRLPOOL,1151.42,2270.00,-34.03,M-SC,96.84,166154.0,19924.0,122140.0,0.92,13.62,73.51,97.15,27.0,0.16,1.31,41.94,X40,NTT,DURABLES
75,SONACOMS,578.05,1006.00,-26.71,M-MC,95.79,94806.0,-6353.0,81239.0,-0.20,-6.28,85.69,74.03,49.0,-0.08,0.75,33.24,AR,BTT,AUTO


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.17
1,25,38.81
2,50,68.23


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    35.00
X40N    15.48
X40     14.32
AR      14.15
XR      10.44
XY25     8.31
X200     1.19
SR       1.11
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.85
M-SC    17.95
X-LC    16.16
H-MC     8.99
M-LC     8.33
H-LC     5.95
X-MC     5.58
M-MC     5.58
X-SC     2.84
L-SC     2.30
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.76,0.41,31.28
BANKS,10.95,-8.57,61.82
IT,9.16,-19.28,85.83
FINANCE,7.57,-15.44,67.53
MISC,7.20,-24.12,77.52
INSURANCE,6.17,1.17,28.62
PAINTS,5.58,-16.41,50.86
AUTO,4.44,-7.68,54.62
ELECTRICAL,4.12,-6.40,60.62


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2917293.0,27
AR,1271416.0,15
XR,1222289.0,15
X40N,1202702.0,18
X40,778984.0,12
SR,203719.0,2
XY25,115901.0,5
X200,67201.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2296137.0,22
M-SC,1962168.0,21
X-LC,693426.0,12
M-MC,561092.0,5
H-MC,467245.0,8
X-SC,348731.0,4
M-LC,323883.0,6
L-SC,307192.0,4
X-MC,298287.0,5


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      880823.0      6
M-SC       XY24      739942.0      6
H-SC       AR        532666.0      3
           X40N      514030.0      7
M-SC       X40N      434478.0      6
           XR        418923.0      5
M-MC       XY24      395845.0      3
X-LC       X40       375166.0      5
H-SC       XR        281974.0      4
M-LC       XY24      202946.0      4
M-SC       X40       192255.0      2
H-LC       AR        191418.0      4
X-LC       AR        180659.0      3
H-MC       XY24      169562.0      2
L-SC       XR        161353.0      2
X-MC       XY24      143609.0      2
H-MC       AR        129264.0      2
X-SC       SR        119802.0      1
           XR        114785.0      1
           X40N      114144.0      2
M-LC       XR        103118.0      1
X-MC       X40N       99966.0      2
M-SC       AR         92653.0      1
H-SC       X40        86644.0      2
M-MC       XR         84008.0      1
M-SC       SR         83917.0      1
L-SC       XY24       82322.0      1
M-MC       AR         81239.0      1
X-LC       XY25       70400.0      3
H-MC       X40        70207.0      2
X-LC       X200       67201.0      1
L-SC       AR         63517.0      1
H-MC       XR         58128.0      1
X-MC       X40        54712.0      1
H-MC       X40N       40084.0      1
H-LC       XY25       27682.0      1
M-LC       XY25       17819.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
